# GOES-R Lightning Data: Max Flash Point Extraction

This notebook processes GOES-R Lightning NetCDF datasets to extract the maximum lightning flash point from each file. It converts the satellite-projected coordinates to geographic coordinates (latitude and longitude), aggregates the maximum flash locations, and exports the results as a GeoJSON file for visualization and further analysis.

The data used in this notebook comes from the **GOES-R Series Geostationary Lightning Mapper (GLM)** Level 2 NetCDF products provided by NOAA/NESDIS. These datasets capture lightning flash information across the Americas with high spatial and temporal resolution.

For more information, visit the official NOAA GLM product page:  
https://www.nesdis.noaa.gov/GOES-R-series/geostationary-lightning-mapper


---

## How to Retrieve GOES-R Lightning Data from NASA Earthdata

You can download GOES-R GLM data from NASA's Earthdata Search portal:

🔗 https://search.earthdata.nasa.gov/

### Steps:
1. Go to [search.earthdata.nasa.gov](https://search.earthdata.nasa.gov/).
2. In the search bar, enter: **"GLM L2 Lightning Detection GOES"** or a more specific product name like `"OR_GLM-L2-LCFA_G16"` or `"GLMF"`.
3. Use the map to narrow the spatial region (if desired).
4. Set your date range in the filter panel.
5. Browse the results and click **Download All** or use the **Customize** button to select specific files.
6. You’ll need a free [Earthdata Login](https://urs.earthdata.nasa.gov/users/new) to access and download files.

The downloaded `.nc` files can then be processed using the script in this notebook.

---

## Script Overview

- Opens GOES-R NetCDF files from a specified folder.
- Identifies the pixel with the maximum total optical energy (a proxy for lightning intensity).
- Converts pixel coordinates from geostationary projection (radians) to latitude/longitude using the correct satellite longitude and geostationary CRS.
- Compiles all maximum flash points into a GeoDataFrame.
- Exports the combined points as a GeoJSON file for easy use in GIS software and web mapping.


---

**Michael Huff**  
GIS Analyst & Developer

*Feel free to reach out for questions or collaboration!*

michaelhuff17@gmail.com


In [ ]:
%pip install xarray netCDF4 geopandas shapely pyproj

import os
import xarray
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS, Transformer

In [ ]:
input_folder = r"E:\GOES-R Lightning Data\2023"
output_file = r"E:\GOES-R Lightning Data\Processed\2023_flashes_processed.geojson"
variable_name = "Total_Optical_energy"
sat_height = 35786023

In [ ]:
def get_satellite_longitude(ds):
    slot = ds.attrs.get('orbital_slot', '').lower()
    if 'east' in slot:
        return -75.0   # GOES-East
    elif 'west' in slot:
        return -137.0  # GOES-West
    else:
        print(f"Warning: Unknown orbital_slot '{slot}', defaulting to GOES-East")
        return -75.0

In [ ]:
records = []
count = 0
wgs84 = CRS.from_epsg(4326)

for file in os.listdir(input_folder):
    if not file.endswith(".nc"):
        continue

    filepath = os.path.join(input_folder, file)
    ds = xarray.open_dataset(filepath)

    if variable_name not in ds:
        continue

    sat_lon = get_satellite_longitude(ds)

    proj_var = ds['goes_imager_projection']
    
    proj_str = (
        f"+proj=geos +h={sat_height} +lon_0={sat_lon} "
        "+sweep=x +a=6378137 +b=6356752.31414 +units=m +no_defs"
    )

    goes_crs = CRS.from_proj4(proj_str)
    transformer = Transformer.from_crs(goes_crs, wgs84, always_xy=True)

    energy_var = ds['Total_Optical_energy']
    energy_raw = energy_var.values.astype(float)

    # Mask fill values (if any)
    fill_value = energy_var.attrs.get('_FillValue', 0)
    energy_raw[energy_raw == fill_value] = np.nan
    energy = energy_raw  # no manual scaling needed, xarray likely decodes automatically

    x = ds['x'].values.astype(float)
    y = ds['y'].values.astype(float)

    valid_pixels = np.count_nonzero(~np.isnan(energy))
    if valid_pixels == 0:
        print(f"No valid energy data in {file}, skipping")
        continue

    max_index = np.unravel_index(np.nanargmax(energy), energy.shape)
    max_value = energy[max_index]

    if np.isnan(max_value) or max_value <= 0:
        print(f"Invalid max energy in {file}, skipping")
        continue

    x_rad = x[max_index[1]]
    y_rad = y[max_index[0]]

    x_meters = x_rad * sat_height
    y_meters = y_rad * sat_height

    lon, lat = transformer.transform(x_meters, y_meters)

    print(f"Processing: {file}")
    
    print(f"Energy max index: {max_index}, energy value: {energy[max_index]}")
    print(f"x at max index: {x[max_index[1]]}, y at max index: {y[max_index[0]]}")
    
    print(f"  Satellite lon: {sat_lon}")
    print(f"  Max energy: {max_value}")
    print(f"  Max index: {max_index}")
    print(f"  x_rad, y_rad: {x_rad}, {y_rad}")
    print(f"  lon, lat: {lon}, {lat}")
    print(f"  Raw energy min: {np.nanmin(energy_raw)}, max: {np.nanmax(energy_raw)}")
    print(f"  Unique values: {np.unique(energy_raw)[:10]}...")

    records.append({
        'filename': file,
        'energy': max_value,
        'geometry': Point(lon, lat)
    })

    count += 1
    print(f"Finished file no. {count}.")


if records:
    gdf = gpd.GeoDataFrame(records, crs="EPSG:4326")
    gdf.to_file(output_file, driver="GEOJSON")


print(f"  Scaled energy min: {np.nanmin(energy)}, max: {np.nanmax(energy)}")
